In [ ]:
LOAD CSV WITH HEADERS FROM ('https://raw.githubusercontent.com/uniaovasko/uniaovasko_vasko/main/project-2-final/data/processed/receitas.csv') AS line
CREATE (:Receita {id: line.id, titulo: line.titulo, regiao: line.regiao})

In [ ]:
CREATE INDEX FOR (r:Receita) ON (r.id)

Para que pudessemos executar nossas querries em um ambiente neo4j online, decidimos filtrar apenas as receitas da região "USA", mas idealmente, usaríamos todas as 45k receitas em nossos dados

In [ ]:
LOAD CSV WITH HEADERS FROM ('https://raw.githubusercontent.com/uniaovasko/uniaovasko_vasko/main/project-2-final/data/processed/ingredientes.csv') AS line
CREATE (:Ingrediente {id: line.id, nome:line.nome, grupo:line.grupo})

In [ ]:
CREATE INDEX FOR (i:Ingrediente) ON (i.id)

In [ ]:
LOAD CSV WITH HEADERS FROM ('https://raw.githubusercontent.com/uniaovasko/uniaovasko_vasko/main/project-2-final/data/processed/ingredientes_receitas.csv') AS line
MATCH (r:Receita {id: line.id_receita})
MATCH (i:Ingrediente {id: line.id_ingrediente})
CREATE (r)-[e:Emprega]->(i)

In [ ]:
MATCH (i1:Ingrediente)<-[:Emprega]-(r:Receita)-[:Emprega]->(i2:Ingrediente)
MERGE (i1)<-[re:Relaciona]->(i2)
ON CREATE SET re.weight = 1
ON MATCH SET re.weight = re.weight + 1

Aqui termina nossa projeção. Podemos inclusive apagar as arestas :Emprega e os nós :Receita se quisermos economizar espaço

Querry para encontrarmos os ingredientes considerados hubs

In [ ]:
MATCH (i:Ingrediente)
WHERE i.grupo <> 'Baking goods'
WITH i, [(i)-[:Relaciona]-() | 1] AS relationships
WITH i, REDUCE(s = 0, rel IN relationships | s + rel) AS degree
ORDER BY degree DESC
LIMIT 20
RETURN i, degree;

Querry para analisar as maiores distancias entre dois ingredientes
Infelizmente, o processamento durou mais de 10 minutos e falhou por tempo limite

In [ ]:
MATCH (i1:Ingrediente), (i2:Ingrediente)
WHERE id(i1) < id(i2)
WITH i1, i2
CALL apoc.algo.dijkstra(i1, i2, 'Relaciona', 'distance') YIELD path, weight
RETURN i1, i2, weight
ORDER BY weight DESC
LIMIT 1;